In [ ]:
import time
import base64
import json
from pathlib import Path

import requests

In [ ]:
workspace_id = notebookutils.runtime.context["currentWorkspaceId"]
folder_name = "acr"
lakehouse_name = "LH_acr"

headers = {"Authorization": f"Bearer {notebookutils.credentials.getToken('pbi')}"}

## Get artifacts from Github repo
https://github.com/Guust-Franssens/acr


In [ ]:
url = "https://api.github.com/repos/guust-franssens/acr/contents/fabric"
response = requests.get(url)
response.raise_for_status()
gh_items = response.json()

## Deploy the items into a folder

In [ ]:
# create the folder
response = requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/folders", headers=headers)
response.raise_for_status()
if folder_name in [folder["displayName"] for folder in response.json()["value"]]:
    folder_id = next(folder["id"] for folder in response.json()["value"] if folder["displayName"] == folder_name)
else:
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/folders"
    response = requests.post(url, json={"displayName": folder_name}, headers=headers)
    response.raise_for_status()
    folder_id = response.json()["id"]

print(folder_id)

In [ ]:
def get_existing_items_in_folder(workspace_id, folder_id):
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items?rootFolderId={folder_id}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()["value"]


In [ ]:
def deploy_lakehouse(lakehouse_name, existing_items, folder_id=None):
    lakehouse = [item for item in existing_items if item["type"] == "Lakehouse" and item["displayName"] == lakehouse_name]
    if lakehouse:
        print(f"⚠️ Lakehouse already exists under name {lakehouse_name}")
        return lakehouse[0]["id"]

    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/lakehouses"
    body = {
        "displayName": lakehouse_name,
        "folderId": folder_id
    }
    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    if response.status_code == 202:
        print(f"✅ Lakehouse deployment initiated, requesting update after {response.headers['Retry-After']}")
        time.sleep(int(response.headers['Retry-After']))
        response = requests.get(response.headers['Location'], headers=headers)
        response.raise_for_status()

    return response.json()["id"]

In [ ]:
def deploy_notebook(notebook_name, notebook_content, existing_items, folder_id=None):
    # Check if the notebook already exists in the folder
    existing_notebook = [item["id"] for item in existing_items if item["type"] == "Notebook" and item["displayName"] == notebook_name]

    body = {
        "definition": {
            "format": "ipynb",
            "parts": [
                {
                    "path": "notebook-content.ipynb",
                    "payload": base64.b64encode(notebook_content),
                    "payloadType": "InlineBase64"
                }
            ]
        }
    }
    if existing_notebook:
        notebook_id = existing_notebook[0]
        url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/notebooks/{notebook_id}/updateDefinition"
        print(f"⚠️ Notebook already exists, updating {notebook_name}...")
    else:
        url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/notebooks"
        body["displayName"] = notebook_name
        if folder_id:
            body["folderId"] = folder_id
        print(f"✅ Creating {notebook_name}")


    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()

In [ ]:
def deploy_pipeline(gh_pipeline, existing_items, folder_id=None):
    pipeline_name = Path(gh_pipeline["name"]).stem

    existing_pipeline = [item["id"] for item in fabric_items if item["type"] == "DataPipeline" and item["displayName"] == pipeline_name]

    response = requests.get(gh_pipeline["download_url"])
    response.raise_for_status()
    pipeline_definition = response.json()

    nb_100_id = next(item["id"] for item in fabric_items if "NB_100_" in item["displayName"])
    nb_200_id = next(item["id"] for item in fabric_items if "NB_200_" in item["displayName"])
    nb_300_id = next(item["id"] for item in fabric_items if "NB_300_" in item["displayName"])

    pipeline_activities = pipeline_definition["definition"]["parts"][0]["payload"]["properties"]["activities"]
    pipeline_activities[0]["typeProperties"] = {"notebookId": nb_100_id, "workspaceId": workspace_id}
    pipeline_activities[1]["typeProperties"] = {"notebookId": nb_200_id, "workspaceId": workspace_id}
    pipeline_activities[2]["typeProperties"] = {"notebookId": nb_300_id, "workspaceId": workspace_id}

    for part in pipeline_definition["definition"]["parts"]:
        part["payload"] = base64.b64encode(json.dumps(part["payload"]).encode("utf-8")).decode("utf-8")
    
    body = {
        "definition": pipeline_definition["definition"]
    }

    if existing_pipeline:
        print(f"⚠️ Pipeline is already deployed under name {pipeline_name}, updating...")
        data_pipeline_id = existing_pipeline[0]
        url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/dataPipelines/{data_pipeline_id}/updateDefinition"
    else:
        print(f"✅ Deploying pipeline: {pipeline_name}")
        url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/dataPipelines"
        body["displayName"] = pipeline_name
        body["folderId"] = folder_id

    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()



In [ ]:
fabric_items = get_existing_items_in_folder(workspace_id, folder_id)
lakehouse_id = deploy_lakehouse(lakehouse_name, fabric_items, folder_id)

gh_notebooks = [item for item in gh_items if item["name"].endswith(".ipynb")]
for gh_notebook in gh_notebooks:
    if gh_notebook["name"].endswith(".ipynb"):
        response = requests.get(gh_notebook["download_url"])
        response.raise_for_status()
        notebook_content = response.content
        notebook_name = Path(gh_notebook["name"]).stem
        deploy_notebook(notebook_name, notebook_content, fabric_items, folder_id=folder_id)

print(f"⌛ Waiting 30 seconds to allow all notebooks to be deployed")
time.sleep(30)

fabric_items = get_existing_items_in_folder(workspace_id, folder_id)
gh_pipeline = next(item for item in gh_items if item["name"].startswith("PL_"))
deploy_pipeline(gh_pipeline, fabric_items, folder_id=folder_id)

for item in fabric_items:
    if item["type"] == "Notebook":
        notebookutils.notebook.updateDefinition(
            name=item["displayName"],
            workspaceId=workspace_id,
            defaultLakehouse=lakehouse_name, 
            defaultLakehouseWorkspace=workspace_id, 
        )
        print(f"✅ Updated notebook {item['displayName']} to default lakehouse to {lakehouse_name}")

## Launch a first data pipeline run performing a full load

In [ ]:
fabric_items = get_existing_items_in_folder(workspace_id, folder_id)
pipeline_id = next(item["id"] for item in fabric_items if item["type"] == "DataPipeline")  
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{pipeline_id}/jobs/instances?jobType=Pipeline"
response = requests.post(url, headers=headers)  
response.raise_for_status()  